In [1]:
import pickle
import numpy as np
import pandas as pd

In [6]:
directory = '/mnt/home/jbielecki1/NEMA/'
modelFileName = '10000000/XGB/xgbEstimators1000Depth6'
fileName = 'NEMA_IQ_384str_N0_1000_COINCIDENCES_PREPARED_part'
max_depth = 6
feature_names = ['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'vol1', 'vol2', 'e1', 'e2', 'dt', 'rX1', 'rY1', 'rZ1', 'volD', 'lorL', 'deg3D', 'deg2D', 'rL', 'eSum']
dataPositiveParts = []

In [7]:
model = pickle.load(open(directory + modelFileName, 'rb'))

In [16]:
def getGOJAFormatPositivePrediction(filePath):
    data = pickle.load(open(filePath, 'rb'))
    codes = {1:1, 2:0, 3:0, 4:0}
    y = data["class"].map(codes)
    x = data.drop(["t1", "t2", "sX1", "sY1", "sZ1", "class", "rError"], axis = 1)
    x.columns = ["f" + str(x) for x in range(20)]
    y_pred_prob = model.predict_proba(x)
    y_pred = y_pred_prob > 0.5
    pPsPredictedPositive = x[y_pred[:,1]]
    dataPositive = data.iloc[list(pPsPredictedPositive.index),:]
    return dataPositive.iloc[:,:16]

In [17]:
for i in range(8):
    dataPositiveParts.append(getGOJAFormatPositivePrediction(directory + fileName + '0' + str(i+1)))

for i in range(8):
    dataPositiveParts.append(getGOJAFormatPositivePrediction(directory + fileName + '1' + str(i)))
    
dataRec = pd.concat(dataPositiveParts)

In [18]:
len(dataRec)

60829205

In [19]:
dataRec.to_csv(directory + 'xgbReconstruction_parts16', sep = "\t", header = False, index = False)